In [1]:
import pandas as pd
import os
import glob
import cudf
import cupy as cp

csv_path = "/home/patwuch/projects/vehicular/data/raw/2024資料/2024.csv"
df = pd.read_csv(csv_path, encoding='Big5')
df


/tmp/ipykernel_3521/4098640339.py:8: DtypeWarning: Columns (95,97) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding='Big5')


,Order,ID,NewID,NewCode,OcDate,Year,Month,Date,Day,DOC,...,JE_1,JE_2,JE_3,DOI_d,PE_d,DQ_d,CT1_d,CT2_d,CT3_d,CT4_d
0,9320705,A2399,11301A2399B0001,11301A2399B00012024-01-0474000,2024-01-04,2024,January,4,Thursday,Weekday,...,Otherwise,NaN,NaN,Non fatal,Restrained,Licensed,Head-on crash,Run-off crash,NaN,Rear end
1,9320706,A2399,11301A2399B0001,11301A2399B00012024-01-0474000,2024-01-04,2024,January,4,Thursday,Weekday,...,Otherwise,Otherwise,Otherwise,Non fatal,Restrained,Licensed,Head-on crash,Run-off crash,NaN,Rear end
2,9320707,A2399,11301A2399B0002,11301A2399B00022024-01-05113500,2024-01-05,2024,January,5,Friday,Weekday,...,Otherwise,Otherwise,Otherwise,Non fatal,Restrained,Licensed,Head-on crash,Run-off crash,NaN,Otherwise
3,9320708,A2399,11301A2399B0002,11301A2399B00022024-01-05113500,2024-01-05,2024,January,5,Friday,Weekday,...,Otherwise,Otherwise,Otherwise,Non fatal,Restrained,Licensed,Head-on crash,Run-off crash,NaN,Otherwise
4,9320709,A2399,11301A2399B0002,11301A2399B00022024-01-05113500,2024-01-05,2024,January,5,Friday,Weekday,...,NaN,NaN,NaN,Non fatal,Restrained,NaN,Head-on crash,Run-off crash,NaN,Otherwise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888520,10209225,BZ100,11312BZ100B0002,11312BZ100B00022024-12-13174249,2024-12-13,2024,December,13,Friday,Weekday,...,Otherwise,Otherwise,Otherwise,Non fatal,Restrained,Licensed,Head-on crash,Run-off crash,NaN,Otherwise
888521,10209226,BZ100,11312BZ100B0002,11312BZ100B00022024-12-13174249,2024-12-13,2024,December,13,Friday,Weekday,...,Otherwise,Otherwise,Otherwise,Non fatal,Restrained,Licensed,Head-on crash,Run-off crash,NaN,Otherwise
888522,10209227,BZ100,11312BZ100B0003,11312BZ100B00032024-12-25171900,2024-12-25,2024,December,25,Wednesday,Weekday,...,Otherwise,Otherwise,Otherwise,Non fatal,Restrained,Licensed,Head-on crash,Run-off crash,NaN,Rear end
888523,10209228,BZ100,11312BZ100B0003,11312BZ100B00032024-12-25171900,2024-12-25,2024,December,25,Wednesday,Weekday,...,NaN,NaN,NaN,Non fatal,NaN,NaN,Head-on crash,Run-off crash,NaN,Rear end


In [2]:
# === Clean and Convert ===
df = df.drop(index=0).reset_index(drop=True)
df['GPS_X'] = pd.to_numeric(df['GPS_X'], errors='coerce')
df['GPS_Y'] = pd.to_numeric(df['GPS_Y'], errors='coerce')
df = df.dropna(subset=['GPS_X', 'GPS_Y'])

# === Load Weather Station CSVs ===
station_dir = "/home/patwuch/projects/vehicular/data/raw/2024資料/全部天氣站_2024/全部天氣站_2024"
station_csvs = glob.glob(os.path.join(station_dir, '*.csv'))
station_all = pd.concat([pd.read_csv(f, encoding='utf-8') for f in station_csvs], ignore_index=True, sort=False)
station_all = station_all.drop(columns=['Unnamed: 27'], errors='ignore')

In [3]:
import re

location_file = "/home/patwuch/projects/vehicular/data/raw/2024資料/77個氣象站經緯度.xlsx"
location_df = pd.read_excel(location_file, usecols=["測站名稱", "經度", "緯度", "測站ID"])
import re
# Function: keep only Chinese characters
def extract_chinese(text):
    if pd.isna(text):
        return ""
    return "".join(re.findall(r'[\u4e00-\u9fff]+', str(text)))

# Clean Chinese-only names
location_df["chinese_name"] = location_df["測站名稱"].apply(extract_chinese)

# Sets for fast lookup
sitename_list = station_all["測站"].astype(str).tolist()

# Attempt substring-based matching
def fuzzy_match(name):
    for site in sitename_list:
        if site and site in name:  # site is contained in chinese_name
            return site
    return None

# Find fuzzy matches
location_df["fuzzy_match"] = location_df["chinese_name"].apply(fuzzy_match)

# Replace 測站名稱 with sitename where fuzzy_match succeeded
location_df.loc[location_df["fuzzy_match"].notna(), "測站名稱"] = \
    location_df.loc[location_df["fuzzy_match"].notna(), "fuzzy_match"]

# For reference, show what was changed
changed_rows = location_df.loc[location_df["fuzzy_match"].notna(), 
                               ["chinese_name", "fuzzy_match", "測站名稱"]]

print("Updated rows (Chinese name → matched sitename):")
print(changed_rows)


Updated rows (Chinese name → matched sitename):
   chinese_name fuzzy_match 測站名稱
0            二林          二林   二林
1            三重          三重   三重
2            三義          三義   三義
3            土城          土城   土城
4      士林科教館氣象站          士林   士林
..          ...         ...  ...
72           頭份          頭份   頭份
73           龍潭          龍潭   龍潭
74           豐原          豐原   豐原
75           關山          關山   關山
76           觀音          觀音   觀音

[77 rows x 3 columns]


In [4]:
for col in location_df.select_dtypes(include=["object"]).columns:
    location_df[col] = (
        location_df[col]
        .astype(str)
        .str.replace(r"[\t\n\r\u3000]", "", regex=True)
        .str.strip()
    )


In [5]:
location_df = location_df.drop(columns=["chinese_name", "fuzzy_match"])

In [6]:
station_all = station_all.merge(location_df, left_on='測站', right_on='測站名稱', how='left')


In [7]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import re

# ---- 1. Convert DMS strings to decimal degrees ----
def maybe_convert_dms(val):
    if isinstance(val, str):
        # Remove any quotes or extra spaces
        val_clean = val.replace('"', '').replace("“", "").replace("”", "").strip()
        # Match DMS pattern: degrees° minutes' seconds
        match = re.match(r'(\d+)[°\s]+(\d+)[\'\s]+([\d.]+)', val_clean)
        if match:
            deg, minute, sec = map(float, match.groups())
            return deg + minute/60 + sec/3600
        else:
            # If it doesn't match DMS, try converting to float directly
            try:
                return float(val_clean)
            except:
                return np.nan
    # If it's already numeric, return as is
    return val

location_df['經度'] = location_df['經度'].apply(maybe_convert_dms)
location_df['緯度'] = location_df['緯度'].apply(maybe_convert_dms)

# ---- 2. Ensure df coordinates are numeric ----
df['GPS_X'] = pd.to_numeric(df['GPS_X'], errors='coerce')
df['GPS_Y'] = pd.to_numeric(df['GPS_Y'], errors='coerce')

# ---- 3. Separate rows with NaN coordinates ----
null_rows = df[df[['GPS_X', 'GPS_Y']].isna().any(axis=1)].copy()
valid_rows = df.dropna(subset=['GPS_X', 'GPS_Y']).copy()

# ---- 4. Drop NaN in location_df coordinates ----
location_clean = location_df.dropna(subset=['經度', '緯度']).copy()

# ---- 5. Build KDTree and find nearest station ----
station_coords = location_clean[['經度', '緯度']].values
tree = cKDTree(station_coords)

event_coords = valid_rows[['GPS_X', 'GPS_Y']].values
distances, indices = tree.query(event_coords, k=1)

# ---- 6. Assign nearest station info ----
valid_rows['nearest_site'] = location_clean.iloc[indices]['測站名稱'].values
valid_rows['site_id'] = location_clean.iloc[indices]['測站ID'].values

# ---- 7. Combine back if you want, keeping null_rows separate ----
df_processed = pd.concat([valid_rows, null_rows], ignore_index=True)

# Now:
# - df_processed contains all rows, with nearest station info where possible
# - null_rows contains rows with invalid coordinates for inspection


In [ ]:
# For each row in df, extract weather metrics from station_all and add as new columns
metrics = [
    'AMB_TEMP', 'CO', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'PM2.5',
    'RAINFALL', 'RH', 'SO2', 'WD_HR', 'WIND_DIREC', 'WIND_SPEED',
    'WS_HR', 'CH4', 'NMHC', 'THC'
]

# Convert date columns to datetime for accurate matching
df_processed['OcDate'] = pd.to_datetime(df['OcDate'])
station_all['日期'] = pd.to_datetime(station_all['日期'])

df_merge = df_processed.copy()
df_merge['Hour'] = df_merge['Hour'].astype(int).astype(str).str.zfill(2)
df_merge['OcDate'] = pd.to_datetime(df_merge['OcDate']).dt.date

# Melt station_all once
station_long = station_all.melt(
    id_vars=['測站ID', '日期', '測項'],
    value_vars=[str(h).zfill(2) for h in range(24)],
    var_name='Hour',
    value_name='value'
)
station_long['日期'] = pd.to_datetime(station_long['日期']).dt.date
station_long = station_long[station_long['測項'].isin(metrics)]

# Merge
df_merged = df_merge.merge(
    station_long,
    left_on=['site_id', 'OcDate', 'Hour'],
    right_on=['測站ID', '日期', 'Hour'],
    how='left'
)

# Pivot metrics into columns
df_final = df_merged.pivot_table(
    index=df_processed.columns,  # keep original df structure
    columns='測項',
    values='value',
    aggfunc='first'
).reset_index()

In [ ]:


# # Melt station_all to long format: one row per siteid, date, hour, metric
# station_long = station_all.melt(
#     id_vars=['測站ID', '日期', '測項'],
#     value_vars=[str(h).zfill(2) for h in range(24)],
#     var_name='Hour',
#     value_name='value'
# )

# station_long = station_long[station_long['測項'].isin(metrics)]
# station_long.set_index(['測站ID', '日期', 'Hour', '測項'], inplace=True)

# def get_weather_value(row, metric):
#     try:
#         siteid = str(row['nearest_siteid'])
#         date = pd.to_datetime(row['OcDate'])
#         hour = str(int(row['Hour'])).zfill(2)
#         return station_long.loc[(siteid, date, hour, metric), 'value']
#     except Exception:
#         return None

# for metric in metrics:
#     df[f'{metric}'] = df.apply(lambda row: get_weather_value(row, metric), axis=1)
